In [141]:
import re

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor

In [142]:
df = pd.read_csv("data60.csv", low_memory=False, parse_dates=['ut_ms'])
df = df[0: 100]

In [143]:
df = df.replace(to_replace='?', value=0)
df = df.apply(pd.to_numeric)
df = df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

In [144]:
df = df.fillna(df.mean())

In [5]:
# df['date'] = pd.to_datetime(df['ut_ms'], unit='ms')
# df.drop('ut_ms', axis=1, inplace=True)

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 518 entries, ut_ms to NPWD2882
dtypes: float64(152), int64(366)
memory usage: 404.8 KB


In [146]:
df

,ut_ms,panelsJm2influx,frontJm2influx,backJm2influx,leftJm2influx,rightJm2influx,upJm2influx,downJm2influx,panelssum1Jm2influx_sum,panelssum1decay09Jm2influx_sum,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
0,1219363260000,1.880060e+09,1.819714e+09,0.0,0.000000e+00,1.115663e+07,0.000000e+00,4.723260e+08,0.000000e+00,0.000000e+00,...,0.000574,0.001529,0.159770,0.005765,0.005284,0.000530,0.000527,0.000764,0.006768,0.003389
1,1219366860000,1.880149e+09,1.819874e+09,0.0,0.000000e+00,1.115169e+07,0.000000e+00,4.723065e+08,1.880060e+09,1.692054e+09,...,0.000559,0.001482,0.044725,0.005597,0.005026,0.000509,0.000502,0.000726,0.006583,0.003262
2,1219370460000,1.856999e+09,1.789827e+09,0.0,4.362176e+06,1.598348e+07,1.448610e+08,3.807518e+08,1.880149e+09,1.692134e+09,...,0.000573,0.001508,0.287059,0.005695,0.005097,0.000514,0.000514,0.000731,0.006578,0.003296
3,1219374060000,1.504105e+09,1.440780e+09,0.0,6.976483e+08,0.000000e+00,2.713091e+08,3.918346e+08,1.856999e+09,1.671299e+09,...,0.000578,0.001521,0.137181,0.005735,0.005248,0.000528,0.000516,0.000748,0.006737,0.003423
4,1219377660000,1.878858e+09,1.817652e+09,0.0,3.823645e+06,9.899647e+06,0.000000e+00,4.806031e+08,1.504105e+09,1.353695e+09,...,0.000566,0.001521,0.060457,0.005760,0.005177,0.000525,0.000526,0.000739,0.006675,0.003377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1219705260000,1.888757e+09,1.039091e+09,0.0,5.731324e-08,5.312175e+06,0.000000e+00,1.157267e+09,1.888664e+09,1.699797e+09,...,0.000569,0.001520,0.002264,0.005705,0.005168,0.000526,0.000516,0.000742,0.006653,0.003336
96,1219708860000,1.888850e+09,1.032164e+09,0.0,5.783011e-08,5.274707e+06,0.000000e+00,1.163519e+09,1.888757e+09,1.699881e+09,...,0.000586,0.001532,0.075603,0.005823,0.005239,0.000535,0.000531,0.000755,0.006841,0.003440
97,1219712460000,1.888943e+09,1.684720e+09,0.0,3.935758e-10,9.380306e+06,0.000000e+00,6.413298e+08,1.888850e+09,1.699965e+09,...,0.000572,0.001529,0.159027,0.005724,0.005107,0.000525,0.000519,0.000741,0.006737,0.003348
98,1219716060000,1.819118e+09,1.766981e+09,0.0,5.770706e+07,1.398330e+07,2.575856e+08,3.204492e+08,1.888943e+09,1.700049e+09,...,0.000567,0.001516,0.189925,0.005657,0.005123,0.000523,0.000519,0.000759,0.006739,0.003388


In [147]:
import seaborn as sns


def pics():
    target = df.iloc[:, 485:]

    for x in target:
        plt.figure(figsize=(15, 10))
        sns.scatterplot(data=df, x='ut_ms', y=x)
        plt.show()

#pics()

# Models

In [148]:
X = df.iloc[:, 0:485]
y = df.iloc[:, 485:]

In [149]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2)

# XGBoost

In [150]:
from xgboost import XGBRegressor

lr = MultiOutputRegressor(XGBRegressor())
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

/Users/mymytre/opt/miniconda3/envs/data-science/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [151]:
lr.score(X_train, y_train)

0.40471997435338575

In [152]:
y_pred

array([[0.0795084 , 0.00151516, 0.17527977, ..., 0.00074248, 0.0067837 ,
        0.00335211],
       [0.09445589, 0.00151516, 0.17544225, ..., 0.00074248, 0.0067837 ,
        0.00335211],
       [0.12127548, 0.00151516, 0.17544225, ..., 0.00074248, 0.50018626,
        0.00335211],
       ...,
       [0.11326853, 0.00151516, 0.17544225, ..., 0.00074248, 0.0067837 ,
        0.00335211],
       [0.19106361, 0.00151516, 0.17527977, ..., 0.00074248, 0.0067837 ,
        0.00335211],
       [0.11667485, 0.00151516, 0.17527977, ..., 0.00074248, 0.0067837 ,
        0.00335211]], dtype=float32)

# GradientBoostingRegressor

In [153]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = MultiOutputRegressor(GradientBoostingRegressor())
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)

In [154]:
gbr.score(X_train, y_train)

0.9955349951775981

In [155]:
y_pred

array([[0.07990053, 0.00153255, 0.17503834, ..., 0.0007472 , 0.00676603,
        0.00339873],
       [0.08933599, 0.00151893, 0.17540622, ..., 0.00073732, 0.00679771,
        0.00340195],
       [0.12136341, 0.00150616, 0.17573681, ..., 0.00074877, 0.67703398,
        0.00340124],
       ...,
       [0.12100184, 0.00153482, 0.17580169, ..., 0.00074596, 0.00687469,
        0.00341852],
       [0.18360805, 0.00150208, 0.17502394, ..., 0.0007412 , 0.00664601,
        0.00333272],
       [0.1223254 , 0.00150889, 0.17535904, ..., 0.0007475 , 0.00657225,
        0.003369  ]])

# RandomForestRegressor

In [156]:
from sklearn.ensemble import RandomForestRegressor

rfm = MultiOutputRegressor(RandomForestRegressor())
rfm.fit(X_train, y_train)
y_pred = rfm.predict(X_test)

In [157]:
rfm.score(X_train, y_train)

0.8543485000456758

In [158]:
y_pred

array([[0.09642369, 0.00153097, 0.17517352, ..., 0.00074587, 0.00673649,
        0.00338828],
       [0.10816573, 0.00151499, 0.17538877, ..., 0.00074244, 0.18416811,
        0.00338404],
       [0.12126097, 0.00150296, 0.17563039, ..., 0.00074696, 0.3894525 ,
        0.00337849],
       ...,
       [0.12489898, 0.00152562, 0.17567264, ..., 0.00074886, 0.01893142,
        0.00338202],
       [0.14400255, 0.00150748, 0.17499085, ..., 0.00073788, 0.00665038,
        0.00332177],
       [0.11894541, 0.00151765, 0.17541326, ..., 0.00073863, 0.01165322,
        0.00330161]])